Passo 1 : importar base de dados
passo 2 : calcular o faturamento da cada loja
passo 3 : calcular a quantidade de produtos vendidos em cada loja
passo 4 : calculer o ticket medio dos produtos de cada loja
passo 5 : enviar e-mail para a diretoria
passo 6 : enviar e-mail para cada loja

In [ ]:
#passo 1, importar base de dados
import pandas as pd #lê arquivos excel
tabela_vendas = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Aula 1/Vendas.xlsx')
display(tabela_vendas)#o display mostra em tabelas, sempre é usado quando se trabalha com python e excel

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600
4,2,2019-01-02,Norte Shopping,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,Center Shopping Uberlândia,Short Listrado,2,102,204
100995,69996,2019-12-26,Center Shopping Uberlândia,Mochila,4,270,1080
100996,69996,2019-12-26,Center Shopping Uberlândia,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,Ribeirão Shopping,Camisa Listrado,1,108,108


In [ ]:
#passo 2: calcular o faturamento de cada loja
faturamento = tabela_vendas[['ID Loja','Valor Final']].groupby('ID Loja').sum()#ele agrupa pelo Id e soma o falor final
faturamento = faturamento.rename(columns={0:'Faturamento'})#é assim que se renomeia uma tabela de uma coluna
faturamento = faturamento.sort_values('Valor Final',ascending=False)#o data frame não aceita un sort() normal, então usa-se o sort_values()
display(faturamento)

,Valor Final
ID Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


In [ ]:
#passo 3 quantidade de produto vendido em cada loja
tabela_quantidade = tabela_vendas[['ID Loja','Quantidade']].groupby('ID Loja').sum()#quando se usa várias colunas, é necessáro usar colchetes duplos
tabela_quantidade = tabela_quantidade.rename(columns={0:'Quantidade de itens vendidos'})
tabela_quantidade = tabela_quantidade.sort_values('ID Loja',ascending=False)
display(tabela_quantidade)

In [ ]:
#ticket medio = faturamento/quantidade
tabela_ticket = (faturamento['Valor Final']/tabela_quantidade['Quantidade']).to_frame()#frame é uma tabela do python
tabela_ticket = tabela_ticket.rename(columns={0:'Ticket'})
display(tabela_ticket)

,Ticket
ID Loja,
Bourbon Shopping SP,194.754598
Center Shopping Uberlândia,193.453228
Iguatemi Campinas,197.248909
Iguatemi Esplanada,198.098019
Norte Shopping,189.923231
Novo Shopping Ribeirão Preto,191.775226
Palladium Shopping Curitiba,189.321307
Parque Dom Pedro Shopping,194.519552
Passei das Águas Shopping,191.345324


In [ ]:
#enviar email-s para diretoria
def enviar_email(Nome,tabela):
  #esse codigo sempre é usado para enviar e-mail do Gmail, basta copia-lo e saber usar
  import smtplib
  import email.message

  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = f'''
  Relatorio semanal das vendas
  Faturamento
  {tabela.to_html()}
  Qualquer dúvida, estou á disposição
  '''#precisa editar
    
  msg = email.message.Message()
  msg['Subject'] = f'Relatorio das vendas para {Nome}' #precisa editar
  
  # Fazer antes (apenas na 1ª vez): Ativar Aplicativos não Seguros.
    # Gerenciar Conta Google -> Segurança -> Aplicativos não Seguros -> Habilitar
  # Caso mesmo assim dê o erro: smtplib.SMTPAuthenticationError: (534,
  # Você faz o login no seu e-mail e depois entra em: https://accounts.google.com/DisplayUnlockCaptcha
  msg['From'] = ''#precisa editar
  msg['To'] = ''#precisa editar
  password = ""#precisa editar
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(corpo_email )
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')
tabela_tudo = faturamento.join(tabela_ticket).join(tabela_quantidade)#todos tem o mesmo Id
enviar_email('Paulo',tabela_tudo)

In [ ]:
#email para cada loja
lojas = tabela_vendas['ID Loja'].unique()#da tabela vendas lá do começo, pego apenas o id loja que nesse caso, é o nome dela, como estão repetidos usa-se o metodo unique()
for loja in lojas:
  tabe_loja = tabela_vendas.loc[tabela_vendas['ID Loja']==loja,['ID Loja','Quantidade', 'Valor Final']]
  resumo_loja = tabe_loja.groupby('ID Loja').sum()
  resumo_loja['Ticket Medio'] = resumo_loja['Valor Final']/resumo_loja['Quantidade']
  display(resumo_loja) 